In [18]:
import sys
import os
import torch
import numpy as np
import matplotlib.pyplot as plt

sys.path.append("..")
from src.nce.cd_cnce import CdCnceCrit
from src.nce.cd_mh import CdMHCrit

from src.noise_distr.conditional_normal import ConditionalMultivariateNormal

from src.models.rbm.rbm import Rbm
from src.data.mnist import MnistDataset

from src.training.model_training import train_model
from src.training.training_utils import no_stopping

from src.experiments.noise_distr_utils import get_nce_noise_distr_par, get_cnce_noise_distr_par
from src.experiments.mnist_exp_utils import initialise_params

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## EXPERIMENTS

In [19]:
# Data specs
num_dims = 28**2

# Experiments specs
num_neg_samples = 2

# Training specs
batch_size = 100
num_epochs = 5

In [20]:
# Check accuracy of model
def rbm_acc(rbm, data_loader, k=1):
    acc = 0
    for i, (y, idx) in enumerate(data_loader, 0):
        y_pred = rbm.sample(y, k=k)
        acc += torch.mean(y_pred == y, dim=-1)
        
    return acc / len(data_loader.dataset)

In [21]:
# Configurations to consider in experiments 

config_conditional_multi = {
    "criterion": CdCnceCrit,
    "label": "cd_cnce",
    "estimate_part_fn": False,
    "conditional_noise_distr": True,
    "mcmc_steps": 1,
}

configs = [config_conditional_multi]

In [22]:
# Get data 
training_data = MnistDataset(train=True, root_dir="src/data/datasets/")
train_loader = torch.utils.data.DataLoader(training_data, batch_size=batch_size, shuffle=True)
                             
test_data = MnistDataset(train=False, root_dir="src/data/datasets/")
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)

In [23]:
# Initialise           
weights, vis_bias, hidden_bias = initialise_params()

# Get noise distr. params

# Unconditional
mu_noise_nce, cov_noise_nce = get_nce_noise_distr_par(training_data.get_full_data())

# Conditional
p_m = Rbm(weights=weights.clone(), vis_bias=vis_bias.clone(), hidden_bias=hidden_bias.clone())
cov_noise_cnce = get_cnce_noise_distr_par(training_data.get_full_data(), num_neg_samples, p_m)

RuntimeError: t() expects a tensor with <= 2 dimensions, but self is 3D

In [ ]:
# Run experiments

for config in configs:

    # Make sure that these are "reinitialised"
    p_m, p_n, criterion = None, None, None

    p_m = Rbm(weights=weights.clone(), vis_bias=vis_bias.clone(), hidden_bias=hidden_bias.clone())
    
    if config["conditional_noise_distr"]:
        p_n = ConditionalMultivariateNormal(cov=cov_noise_cnce)
    else:
        p_n = MultivariateNormal(mu=mu_noise_nce, cov=cov_noise_nce)

    if config["mcmc_steps"] is not None:
        criterion = config["criterion"](p_m, p_n, num_neg_samples, config["mcmc_steps"])
    else:
        criterion = config["criterion"](p_m, p_n, num_neg_samples)

    save_dir = None
    _ = train_model(criterion, error_metric, train_loader, save_dir, neg_sample_size=num_neg_samples, num_epochs=num_epochs,
                    decaying_lr=True, stopping_condition=no_stopping)
    
    # Check test accuracy of model
    acc = rbm_acc(p_m test_loader):
    print("Model accuracy: {}".format(acc))
    
    # Save model
    torch.save(p_m.state_dict(), "res/params_" + config["label"])


In [ ]:
# Visualise results

num_samples = 4

p_m = Rbm(weights=weights.clone(), vis_bias=vis_bias.clone(), hidden_bias=hidden_bias.clone())
for config in configs:
    p_m.load_state_dict(torch.load("res/params_" + config["label"]))
    y_pred = p_m.sample(torch.randn((num_samples, num_dims)), k=1) # Eller hur fungerar detta?
    
    plt.imshow(np.transpose(torchvision.utils.make_grid(y_pred.reshape(-1, 1, 28, 28)).numpy(), (1, 2, 0)))
    plt.title(config["label"])
    plt.show()
    
